In [1]:
# pip install reportlab

In [2]:
import json
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Preformatted
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
import os

In [3]:
output_dir = "output"

with open(os.path.join(output_dir, "repo_scan_report.json")) as f:
    repo_data = json.load(f)

with open(os.path.join(output_dir, "microservice_splits.json")) as f:
    microservices = json.load(f)

with open(os.path.join(output_dir, "file_descriptions.json")) as f:
    file_descriptions = json.load(f)

with open(os.path.join(output_dir, "new_microservice_structure.txt"), encoding="utf-8") as f:
    folder_structure = f.read()

In [4]:
folder_structure = folder_structure.replace("📁", "")
folder_structure = folder_structure.replace("└──", "--")

In [5]:
styles = getSampleStyleSheet()
styleN = styles["Normal"]
styleH = styles["Heading1"]

wrap_style = ParagraphStyle(
    name="Wrapped",
    fontName="Helvetica",
    fontSize=9,
    leading=11
)

def create_table(data, col_names, col_widths=None):
    table_data = [col_names] + data
    wrapped_data = [
        [Paragraph(str(cell), wrap_style) for cell in row]
        for row in table_data
    ]
    
    table = Table(wrapped_data, colWidths=col_widths, repeatRows=1)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.grey),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
    ]))
    return table

In [6]:
pdf_path = os.path.join(output_dir, "microservice_report.pdf")
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
story = []

In [7]:
repo_name = repo_data.get("repo", "Unknown Repo")
story.append(Paragraph(f"<b>Repository:</b> {repo_name}", styleN))
story.append(Spacer(1, 12))

In [8]:
story.append(Paragraph("📂 Repository Files", styleH))
repo_table_data = [[file["name"], file["size"], file["path"]] for file in repo_data["files"]]
story.append(create_table(repo_table_data, ["Name", "Size (bytes)", "Path"], col_widths=[150, 80, 280]))
story.append(Spacer(1, 20))

In [9]:
story.append(Paragraph("🔁 Microservice Splits", styleH))
splits_table_data = [[k, ", ".join(v)] for k, v in microservices.items()]
story.append(create_table(splits_table_data, ["Original File", "Split Files"], col_widths=[150, 360]))
story.append(Spacer(1, 20))

In [10]:
story.append(Paragraph("🧾 File Descriptions", styleH))
desc_table_data = [[k, v] for k, v in file_descriptions.items()]
story.append(create_table(desc_table_data, ["File Name", "Description"], col_widths=[150, 360]))
story.append(Spacer(1, 20))

In [11]:
story.append(Paragraph("🗂️ New Microservice Structure", styleH))
story.append(Spacer(1, 8))
story.append(Preformatted(folder_structure, style=wrap_style))  # ✅ This preserves formatting

In [12]:
doc.build(story)
print(f"[✅] PDF report generated at: {pdf_path}")

[✅] PDF report generated at: output\microservice_report.pdf
